<a href="https://colab.research.google.com/github/salmenhsairi/EndOfStudiesProjectNotebooks/blob/main/SpacySpanCat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! pip install spacy-transformers

In [ ]:
# ! sudo -H python -m pip install spacy-experimental

In [ ]:
! pip install -U pip setuptools wheel
# ! pip install -U spacy
# ! python -m spacy download en_core_web_lg

In [ ]:
! pip install spacy==3.2

In [ ]:
# ! python -m spacy download en_core_web_lg
! python -m spacy download en_core_web_sm
# ! python -m spacy download fr_core_news_lg

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# import spacy
# spacy.prefer_gpu()

## Unzip exported span categorizer data

In [ ]:
! unzip *.zip
# # ! mv *.txt spans_key.txt
# # ! mv *.spacy data.spacy
! rm *.zip

Archive:  train-big-project-spancat.zip
 extracting: fcf23529-e37b-4d2f-ae1d-f56f8dd1b9de/1de63575-f2a6-4fed-b04e-05dcf85100db.spacy  
 extracting: fcf23529-e37b-4d2f-ae1d-f56f8dd1b9de/spans_key.txt  
 extracting: 9fdbeb2c-f6a5-4277-8658-35bc14f7f504/spans_key.txt  
 extracting: 9fdbeb2c-f6a5-4277-8658-35bc14f7f504/6686c5aa-2b1d-41f9-a3aa-227e684e5e4f.spacy  


## Upload and Update base_config file

* export base_config file from https://spacy.io/usage/training#config 
* update the spans_key field within the base_config file <br>  to be the same as the one found in the exported /content/spans_key.txt file 

## Create the config file for the model training with the command line below
* fill the config base file for our SpanCat Training task
* config file should contains all the information required for SpanCat pipeline training 

In [ ]:
! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
TRAIN_TEST_SPLIT = 0.2

In [ ]:
import spacy
from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin

nlp = spacy.blank("en")
train_docbin = DocBin().from_disk("/content/train_data.spacy")
dev_docbin = DocBin().from_disk("/content/dev_data.spacy")
train_docs = list(train_docbin.get_docs(nlp.vocab))
dev_docs = list(dev_docbin.get_docs(nlp.vocab))
for docs in [train_docs,dev_docs]:
  for doc in docs:
      doc.spans['sc'] = list(doc.ents)
      doc.spans.pop('grit')
# train_docs, dev_docs = train_test_split(docs,test_size=TRAIN_TEST_SPLIT,shuffle=True,random_state=42)
DocBin(docs=train_docs).to_disk("/content/train_data.spacy")
DocBin(docs=dev_docs).to_disk("/content/dev_data.spacy")

In [ ]:
list(dict(dev_docs[0].spans).keys())[0]

'grit'

In [ ]:
# docs = list(docbin.get_docs(nlp.vocab))[:3]
# train_docs, dev_docs = train_test_split(docs,test_size=0.5,shuffle=True,random_state=42)

## Start the Training

In [ ]:
# the train data is the same as the validation data ! 
# we should consider having train.spacy and valid.spacy at the export
# ! python -m spacy train --gpu-id 0 config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./dev_data.spacy
! python -m spacy train \
config.cfg \
--output ./output \
--paths.train ./train_data.spacy \
--paths.dev ./dev_data.spacy \
--training.max_steps='400' \
# --components.spancat.spans_key="grit" \
# --training.score_weights.spans_sc_f="1.0" \
# --training.score_weights.spans_sc_p="0.0" \
# --training.score_weights.spans_sc_r="0.0" \
# --training.score_weights.spans_grit_f="null" \
# --training.score_weights.spans_grit_p="null" \
# --training.score_weights.spans_grit_r="null" 

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-07-07 12:36:22,685] [INFO] Set up nlp object from config
[2022-07-07 12:36:22,699] [INFO] Pipeline: ['tok2vec', 'spancat']
[2022-07-07 12:36:22,704] [INFO] Created vocabulary
[2022-07-07 12:36:22,706] [INFO] Finished initializing nlp object
[2022-07-07 12:36:34,348] [INFO] Initialized pipeline components: ['tok2vec', 'spancat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'spancat']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS SPANCAT  SPANS_GRIT_F  SPANS_GRIT_P  SPANS_GRIT_R  SCORE 
---  ------  ------------  ------------  ------------  ------------  ------------  ------
  0       0        597.00      15340.38          0.67          0.34         37.26    0.01
  0       1        417.69      10631.32          0.77          0.39         29.62    0.01
  0       2

In [ ]:
import spacy
from spacy.tokens import DocBin
span_cat_nlp = spacy.load('/content/output/model-best')
docbin = DocBin().from_disk("/content/dev_data.spacy")
nlp = spacy.blank("en")
actual_docs = list(docbin.get_docs(nlp.vocab))

In [ ]:
# ! python /content/scripts/evaluate.py /content/output/model-best /content/dev_data.spacy --output /content/metrics.json --spans-key sc

In [ ]:
from spacy.cli.evaluate import evaluate
result = evaluate(
    '/content/output/model-best',
    '/content/dev_data.spacy',
    output='/content/metrics.json',
    use_gpu=-1,
    # gold_preproc=False,
    # displacy_path=None,
    # displacy_limit=25,
    # silent=False,
    # spans_key="grit",
)

/usr/local/lib/python3.7/dist-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  model_version=meta["version"],


In [ ]:
result

{'spans_sc_f': 0.755240303733006,
 'spans_sc_p': 0.8140560674377573,
 'spans_sc_per_type': {'CERTIFICATE': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'CHARACTERISTIC': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'COMPANY': {'f': 0.6245136186770428,
   'p': 0.8403141361256544,
   'r': 0.4969040247678019},
  'DATE': {'f': 0.8082706766917293,
   'p': 0.8509234828496042,
   'r': 0.7696897374701671},
  'EDUCATION': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'EMAIL': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'INDUSTRY': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'JOBTITLE': {'f': 0.7709163346613545,
   'p': 0.8696629213483146,
   'r': 0.6923076923076923},
  'LANGUAGES': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'LOCATION': {'f': 0.7782312925170068,
   'p': 0.8827160493827161,
   'r': 0.6958637469586375},
  'NAME': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'NUMBER': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'SKILL': {'f': 0.7999519980799232,
   'p': 0.8038104425419028,
   'r': 0.7961304192045862},
  'TIME': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'URL': {'f': 0.0, '

In [ ]:
# choose index from 1 -> len(docs)-1 to test the model with
idx = 5
# doc = span_cat_nlp(docs[idx])
# for doc in actual_docs:
#     doc.spans['sc'] = doc.spans['grit']
#     doc.spans.pop('grit')
doc = actual_docs[idx]
spacy.displacy.render(doc,style='span',jupyter=True)

In [ ]:
actual_docs_text = [doc.text for doc in actual_docs]

In [ ]:
text = actual_docs_text[idx]
text_doc = span_cat_nlp(text)
# text_doc.spans['sc'] = text_doc.spans['grit']
# text_doc.to_json()['spans']
spacy.displacy.render(text_doc,style='span',jupyter=True)

In [ ]:
len(text_doc.to_json()['spans']['sc'])

492

In [ ]:
len(text_doc.spans['sc'])

492

In [ ]:
span_objects = text_doc.to_json()['spans']['sc']
span_tokens = text_doc.spans['sc']
full_spans = []
for (span_object,span_token) in zip(span_objects,span_tokens):
  full_span = span_object
  full_span['tokens'] = span_token
  full_spans.append(full_span)
  

In [ ]:
full_spans

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# save trained pipline
span_cat_nlp.to_disk("/content/drive/MyDrive/Colab Notebooks/Work/Spacy Models/span_cat_model")

In [ ]:
import gc
gc.collect()

88

In [ ]:
# span_cat_nlp.config['components']['spancat']['spans_key']
# list(dict(text_doc.spans).keys())[0]

In [ ]:
# nlp = spacy.blank("fr")
# spancat = nlp.add_pipe("/content/drive/MyDrive/Colab Notebooks/Work/Spacy Models/span_cat_invoice1.0")
# spancat.predict([doc])

In [ ]:
# spancat = span_cat_nlp.get_pipe('spancat')
# scores = spancat.predict([text_doc])

In [ ]:
# spancat.set_annotations([text_doc], scores)

In [ ]:
# text_doc.spans['grit'].attrs["scores"]

In [ ]:
# output_spans = text_doc.spans['sc']

In [ ]:
text_doc.spans

{'sc': [finding, Service, communication, Tech, Prof, finding, Service, communication, Agile development, fixing defects, performance issues, Cloud Computing, Agile development, fixing defects, performance issues, Cloud Computing, Service Oriented Architecture, Service Oriented Architecture, AWS/Azure/IBM Cloud, AWS/Azure/IBM Cloud]}

In [ ]:
spacy.displacy.render(text_doc,style='span',jupyter=True)

In [ ]:
text_doc.to_json().keys()

dict_keys(['text', 'tokens', 'spans'])

In [ ]:
from spacy.training import Example
# predicted_docs = [span_cat_nlp(text) for text in actual_docs_text]
examples = [Example(predicted_doc, actual_doc) for (predicted_doc, actual_doc) in zip(actual_docs, actual_docs)]

In [ ]:
# span_cat_nlp.evaluate(examples)

In [ ]:
# span_cat_nlp.evaluate(docs)

In [ ]:
! python -m spacy debug config config.cfg


============================= Config validation =============================
✘ Config validation error
disabled                  field required
tokenizer                 field required
before_creation           field required
after_creation            field required
after_pipeline_creation   field required

{'lang': 'en', 'pipeline': ['tok2vec', 'spancat'], 'batch_size': 1000}

If your config contains missing values, you can run the 'init fill-config'
command to fill in all the defaults, if possible:

python -m spacy init fill-config config.cfg config.cfg 



In [ ]:
from spacy.util import fix_random_seed
from spacy.language import Language
from spacy.training import Example
SPAN_KEY = 'grit'

In [ ]:
TRAIN_DATA = [
    ("Who is Shaka Khan?", {"spans": {SPAN_KEY: [(7, 17, "PERSON")]}}),
    (
        "I like London and Berlin.",
        {"spans": {SPAN_KEY: [(7, 13, "LOC"), (18, 24, "LOC")]}},
    ),
]

In [ ]:
def make_get_examples(nlp):
    train_examples = []
    for t in TRAIN_DATA:
        eg = Example.from_dict(nlp.make_doc(t[0]), t[1])
        train_examples.append(eg)
    def get_examples():
        return train_examples
    return get_examples

In [ ]:
def test_simple_train():
    fix_random_seed(0)
    nlp = Language()
    spancat = nlp.add_pipe("spancat", config={"spans_key": SPAN_KEY})
    get_examples = make_get_examples(nlp)
    nlp.initialize(get_examples)
    sgd = nlp.create_optimizer()
    assert len(spancat.labels) != 0
    for i in range(40):
        losses = {}
        nlp.update(list(get_examples()), losses=losses, drop=0.1, sgd=sgd)
    doc = nlp("I like London and Berlin.")
    assert doc.spans[spancat.key] == doc.spans[SPAN_KEY]
    assert len(doc.spans[spancat.key]) == 2
    assert doc.spans[spancat.key][0].text == "London"
    scores = nlp.evaluate(get_examples(),batch_size=1)
    assert f"spans_{SPAN_KEY}_f" in scores
    assert scores[f"spans_{SPAN_KEY}_f"] == 1.0
    return nlp.config

In [ ]:
conf = test_simple_train()

In [ ]:
conf

{'components': {'spancat': {'factory': 'spancat',
   'max_positive': None,
   'model': {'@architectures': 'spacy.SpanCategorizer.v1',
    'reducer': {'@layers': 'spacy.mean_max_reducer.v1', 'hidden_size': 128},
    'scorer': {'@layers': 'spacy.LinearLogistic.v1', 'nI': None, 'nO': None},
    'tok2vec': {'@architectures': 'spacy.Tok2Vec.v1',
     'embed': {'@architectures': 'spacy.MultiHashEmbed.v1',
      'attrs': ['ORTH', 'PREFIX', 'SUFFIX', 'SHAPE'],
      'include_static_vectors': False,
      'rows': [5000, 2000, 1000, 1000],
      'width': 96},
     'encode': {'@architectures': 'spacy.MaxoutWindowEncoder.v1',
      'depth': 4,
      'maxout_pieces': 3,
      'width': 96,
      'window_size': 1}}},
   'scorer': {'@scorers': 'spacy.spancat_scorer.v1'},
   'spans_key': 'grit',
   'suggester': {'@misc': 'spacy.ngram_suggester.v1', 'sizes': [1, 2, 3]},
   'threshold': 0.5}},
 'corpora': {'dev': {'@readers': 'spacy.Corpus.v1',
   'augmenter': None,
   'gold_preproc': False,
   'limit': 

In [ ]:
test_simple_train()

In [ ]:
train_docs[0].to_json()['spans']

{'grit': [{'end': 45, 'kb_id': '', 'label': 'SKILLS', 'start': 37},
  {'end': 59, 'kb_id': '', 'label': 'SKILLS', 'start': 50},
  {'end': 90, 'kb_id': '', 'label': 'SKILLS', 'start': 78},
  {'end': 108, 'kb_id': '', 'label': 'SKILLS', 'start': 95},
  {'end': 154, 'kb_id': '', 'label': 'SKILLS', 'start': 136},
  {'end': 316, 'kb_id': '', 'label': 'SKILLS', 'start': 312},
  {'end': 326, 'kb_id': '', 'label': 'SKILLS', 'start': 318},
  {'end': 365, 'kb_id': '', 'label': 'SKILLS', 'start': 332},
  {'end': 437, 'kb_id': '', 'label': 'SKILLS', 'start': 410},
  {'end': 545, 'kb_id': '', 'label': 'SKILLS', 'start': 518},
  {'end': 648, 'kb_id': '', 'label': 'SKILLS', 'start': 635},
  {'end': 732, 'kb_id': '', 'label': 'SKILLS', 'start': 723},
  {'end': 820, 'kb_id': '', 'label': 'SKILLS', 'start': 801},
  {'end': 865, 'kb_id': '', 'label': 'SKILLS', 'start': 840},
  {'end': 1347, 'kb_id': '', 'label': 'EXPERIENCE', 'start': 1277},
  {'end': 1478, 'kb_id': '', 'label': 'EXPERIENCE', 'start': 13

In [ ]:
doc = train_docs[0]

In [ ]:
span = doc.spans['grit'][0]

In [ ]:
span.end_char

45

In [ ]:
doc = span.as_doc()

In [ ]:
span

Planning

In [ ]:
doc.to_json()

{'ents': [{'end': 8, 'label': 'SKILLS', 'start': 0}],
 'sents': [{'end': 8, 'start': 0}],
 'text': 'Planning ',
 'tokens': [{'end': 8, 'id': 0, 'start': 0}]}

In [ ]:
doc.to_json()['tokens']

[{'end': 3, 'id': 0, 'start': 0},
 {'end': 9, 'id': 1, 'start': 4},
 {'end': 19, 'id': 2, 'start': 10},
 {'end': 24, 'id': 3, 'start': 20},
 {'end': 36, 'id': 4, 'start': 25},
 {'end': 45, 'id': 5, 'start': 37},
 {'end': 49, 'id': 6, 'start': 46},
 {'end': 59, 'id': 7, 'start': 50},
 {'end': 60, 'id': 8, 'start': 59},
 {'end': 68, 'id': 9, 'start': 61},
 {'end': 71, 'id': 10, 'start': 69},
 {'end': 77, 'id': 11, 'start': 72},
 {'end': 90, 'id': 12, 'start': 78},
 {'end': 94, 'id': 13, 'start': 91},
 {'end': 108, 'id': 14, 'start': 95},
 {'end': 109, 'id': 15, 'start': 108},
 {'end': 118, 'id': 16, 'start': 110},
 {'end': 132, 'id': 17, 'start': 119},
 {'end': 135, 'id': 18, 'start': 133},
 {'end': 147, 'id': 19, 'start': 136},
 {'end': 154, 'id': 20, 'start': 148},
 {'end': 157, 'id': 21, 'start': 155},
 {'end': 162, 'id': 22, 'start': 158},
 {'end': 166, 'id': 23, 'start': 163},
 {'end': 171, 'id': 24, 'start': 167},
 {'end': 180, 'id': 25, 'start': 172},
 {'end': 183, 'id': 26, 'star

In [ ]:
doc.to_json()['spans']

{'grit': [{'end': 45, 'kb_id': '', 'label': 'SKILLS', 'start': 37},
  {'end': 59, 'kb_id': '', 'label': 'SKILLS', 'start': 50},
  {'end': 90, 'kb_id': '', 'label': 'SKILLS', 'start': 78},
  {'end': 108, 'kb_id': '', 'label': 'SKILLS', 'start': 95},
  {'end': 154, 'kb_id': '', 'label': 'SKILLS', 'start': 136},
  {'end': 316, 'kb_id': '', 'label': 'SKILLS', 'start': 312},
  {'end': 326, 'kb_id': '', 'label': 'SKILLS', 'start': 318},
  {'end': 365, 'kb_id': '', 'label': 'SKILLS', 'start': 332},
  {'end': 437, 'kb_id': '', 'label': 'SKILLS', 'start': 410},
  {'end': 545, 'kb_id': '', 'label': 'SKILLS', 'start': 518},
  {'end': 648, 'kb_id': '', 'label': 'SKILLS', 'start': 635},
  {'end': 732, 'kb_id': '', 'label': 'SKILLS', 'start': 723},
  {'end': 820, 'kb_id': '', 'label': 'SKILLS', 'start': 801},
  {'end': 865, 'kb_id': '', 'label': 'SKILLS', 'start': 840},
  {'end': 1347, 'kb_id': '', 'label': 'EXPERIENCE', 'start': 1277},
  {'end': 1478, 'kb_id': '', 'label': 'EXPERIENCE', 'start': 13

In [ ]:
doc.text[1348:1478]

'3+ years of people management experience, or relevant experience demonstrating readiness to step up into a people management role.'

In [ ]:
span = doc.spans['grit'][15]

In [ ]:
span.end_char

1478